# Práctica Calificada 02
## Predicción de tarifas de taxis

<span style="color:red">
Abrir el cuaderno en Google Colab y ejecutar las celdas nuevamente.
</span>.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
drive.mount('/gdrive')
%cd /gdrive/MyDrive/AIProjects/PC2/Data/PracticaCalificada2-CC421/
#%ls

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/AIProjects/PC2/Data/PracticaCalificada2-CC421


In [6]:
import pandas as pd
import numpy as np

In [ ]:
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

#### Importar data total

In [7]:
dtypes = {
      'key': 'object',
      'fare_amount': 'float64',
      'pickup_datetime': 'object',
      'pickup_longitude': 'float64', 
      'pickup_latitude': 'float64',
      'dropoff_longitude': 'float64',
      'dropoff_latitude': 'float64',
      'passenger_count': 'int64',
      
    
}
df = pd.read_csv("train.csv", dtype=dtypes, nrows=1000000)
df = df[['key', 'pickup_datetime', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'passenger_count', 'fare_amount']]
df.head()

,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,40.721319,-73.844311,40.712278,-73.841610,1,4.5
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,40.711303,-74.016048,40.782004,-73.979268,1,16.9
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,40.761270,-73.982738,40.750562,-73.991242,2,5.7
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,40.733143,-73.987130,40.758092,-73.991567,1,7.7
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,40.768008,-73.968095,40.783762,-73.956655,1,5.3


In [ ]:
df.info()

### Primera revisión de la data completa

#### Valores duplicados

In [8]:
%time
duplicateRowsDF = df.loc[df.duplicated(['key'], keep=False), :]
duplicateRowsDF

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount


In [9]:
df['key'][55423855] =  '2014-12-12 11:33:00.00000016'

In [10]:
duplicateRowsDF = df.loc[df.duplicated(['key'], keep=False), :]
duplicateRowsDF.shape[0]

0

#### Valores NaN

In [11]:
df.isna().sum()

key                   0
pickup_datetime       0
pickup_latitude       0
pickup_longitude      0
dropoff_latitude     10
dropoff_longitude    10
passenger_count       0
fare_amount           0
dtype: int64

In [13]:
df = df.dropna()

#### Coordenadas

Se analizará la data con el objetivo de eliminar las filas con coordenadas ubicadas en el océano.


In [14]:
!pip install global-land-mask

     |████████████████████████████████| 1.8MB 6.4MB/s 


In [15]:
from global_land_mask import globe as gl

In [16]:
df = df.dropna()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999990 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   key                999990 non-null  object 
 1   pickup_datetime    999990 non-null  object 
 2   pickup_latitude    999990 non-null  float64
 3   pickup_longitude   999990 non-null  float64
 4   dropoff_latitude   999990 non-null  float64
 5   dropoff_longitude  999990 non-null  float64
 6   passenger_count    999990 non-null  int64  
 7   fare_amount        999990 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 68.7+ MB


In [18]:
df = df[df['pickup_latitude'].values > -90]
df = df[df['pickup_latitude'].values < 90]
df = df[df['dropoff_latitude'].values > -90]
df = df[df['dropoff_latitude'].values < 90]

df = df[df['pickup_longitude'].values > -180]
df = df[df['pickup_longitude'].values < 180]
df = df[df['dropoff_longitude'].values > -180]
df = df[df['dropoff_longitude'].values < 180]

In [21]:
pla = df[['pickup_latitude']].to_numpy()
plon = df[['pickup_longitude']].to_numpy()
dla = df[['dropoff_latitude']].to_numpy()
dlon = df[['dropoff_longitude']].to_numpy()

#gl.is_ocean(pla,plon)
#df.loc[gl.is_ocean(pla,plon), :]

df = df.drop(df.loc[gl.is_ocean(pla,plon)].index)

In [ ]:
# dfs[['pickup_latitude', 'pickup_longitude']] = dfs.apply(lambda x: ocean_values_to_nan(x['pickup_latitude'],x['pickup_longitude']), axis=1 )
# dfs[['dropoff_latitude', 'dropoff_longitude']] = dfs.apply(lambda x: ocean_values_to_nan(x['dropoff_latitude'],x['dropoff_longitude']) , axis=1)

In [22]:
df.isna().sum()

key                  0
pickup_datetime      0
pickup_latitude      0
pickup_longitude     0
dropoff_latitude     0
dropoff_longitude    0
passenger_count      0
fare_amount          0
dtype: int64

In [ ]:
#df.to_csv('trainsampleprep.csv')

In [23]:
df.head()

,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,40.721319,-73.844311,40.712278,-73.841610,1,4.5
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,40.711303,-74.016048,40.782004,-73.979268,1,16.9
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,40.761270,-73.982738,40.750562,-73.991242,2,5.7
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,40.733143,-73.987130,40.758092,-73.991567,1,7.7
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,40.768008,-73.968095,40.783762,-73.956655,1,5.3


#### Rango de valores

Ahora verificamos que los datos cumplan las restricciones de rango correspondientes.

In [24]:
df.describe()

NameError: ignored

Observamos que el mínimo valor de la columna de tarifa (fare_amount) es -6.5. Necesitamos que los valores de la tarifa sean valores mayores o iguales a 0.

In [26]:
df = df[df['fare_amount'] >= 0]

También notamos valores atípicos en la cantidad de pasajeros, por ejemplo un viaje con 208 pasajeros. De acuerdo a las leyes de tránsito de Nueva York como máximo pueden haber 6 personas. Por lo tanto, las filas con cantidades de pasajeros mayores a 6 serán eliminadas.

In [27]:
df = df[df['passenger_count'] <= 6]

In [28]:
df.to_csv('trainsampleprep.csv')

#### Normalización

In [29]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfn = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
dfn.head()

ValueError: ignored

In [ ]:
dff = pd.DataFrame(af, columns = ['key','fare_amount','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count'])
dff.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0.096626,0.256750,0.094779,0.486964,0.509565,0.487987,0.520137,0.004808
1,0.095404,0.254083,0.093577,0.486967,0.509570,0.487992,0.520145,0.014423
2,0.193632,0.259417,0.189024,0.486962,0.509562,0.487989,0.520142,0.004808
3,0.841936,0.255000,0.810950,0.486965,0.509566,0.487990,0.520143,0.009615
4,0.384110,0.260083,0.370704,0.486959,0.509561,0.487990,0.520142,0.004808


In [30]:
import xgboost
from sklearn.model_selection import train_test_split

predictors = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']
salida = ['fare_amount']
X = df[predictors]
y = df[salida]

X_train, X_val, y_train, y_val = train_test_split(X,y)

In [34]:
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train,
 eval_set=[(X_val, y_val)])
y_pred = xgb_reg.predict(X_val)



[18:08:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:13.491
[1]	validation_0-rmse:12.4933
[2]	validation_0-rmse:11.6204
[3]	validation_0-rmse:10.856
[4]	validation_0-rmse:10.1883
[5]	validation_0-rmse:9.61231
[6]	validation_0-rmse:9.10681
[7]	validation_0-rmse:8.68104
[8]	validation_0-rmse:8.30876
[9]	validation_0-rmse:7.97683
[10]	validation_0-rmse:7.69936
[11]	validation_0-rmse:7.45036
[12]	validation_0-rmse:7.24844
[13]	validation_0-rmse:7.06655
[14]	validation_0-rmse:6.90126
[15]	validation_0-rmse:6.75596
[16]	validation_0-rmse:6.63845
[17]	validation_0-rmse:6.52995
[18]	validation_0-rmse:6.41596
[19]	validation_0-rmse:6.33234
[20]	validation_0-rmse:6.26707
[21]	validation_0-rmse:6.20488
[22]	validation_0-rmse:6.13617
[23]	validation_0-rmse:6.09048
[24]	validation_0-rmse:6.051
[25]	validation_0-rmse:5.98942
[26]	validation_0-rmse:5.95023
[27]	validation_0-rmse:5.89605
[28]	validation_0-r

In [40]:
score_te =xgb_reg.score(X_val, y_val)
print('Accuracy Score: {}'.format(score_te))

Accuracy Score: 0.7259844084968724


In [35]:
from sklearn.metrics import mean_squared_error
error=mean_squared_error(y_val, y_pred)
print(error)

[26.32930848]


In [36]:
import numpy as np
for val,pred in zip(y_val['fare_amount'][:10],y_pred[:10]):
  print("{} vs {}".format(pred,val))

8.754800796508789 vs 8.9
12.147394180297852 vs 12.9
8.013571739196777 vs 8.1
8.013571739196777 vs 6.9
8.14824104309082 vs 6.5
50.237457275390625 vs 45.0
8.089200973510742 vs 5.3
11.712468147277832 vs 14.5
27.108022689819336 vs 18.5
15.354700088500977 vs 16.9
